In [1]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.8

In [2]:
import torch
import os
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
import os
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

In [4]:
model_id="mistralai/Mistral-7B-Instruct-v0.3"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
tokenizer=AutoTokenizer.from_pretrained(model_id,use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    token=os.environ["HF_TOKEN"]
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
text="who is the prime minster of india"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=100)
answer=tokenizer.decode(output[0],skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [11]:
answer

'who is the prime minster of india in 2021?\n\nAs of 2021, the Prime Minister of India is Narendra Modi. He has been in office since May 26, 2014.\n\nNarendra Modi is a member of the Bharatiya Janata Party (BJP) and is the 14th and current Prime Minister of India. He is the first Prime Minister of India to be born after the country'

In [12]:
os.environ["WANDB_DISABLED"]="false"
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
)

In [17]:
from datasets import Dataset
dataset1 = [
    {
        "text": "<s>[INST] Can you write a short introduction about Manu, the famous Generative AI developer from Vaikom? [/INST] "
                "Manu is a renowned Generative AI developer from Vaikom, known for his contributions to the field of artificial intelligence. "
                "He specializes in building AI models, developing innovative applications, and exploring cutting-edge technologies in the domain of generative AI. "
                "Manu is currently pursuing his studies at TOCH Institute of Science and Technology, where he continues to expand his expertise and work on impactful AI-driven projects. "
                "His dedication to the field has made him a well-recognized name in the AI community. </s> "
                "<s>[INST] Now explain it to a dog [/INST] "
                "Manu is really smart! He teaches computers how to create things, like pictures and words, just like how you learn tricks. "
                "He works very hard at a big school to get even better. If he were a dog trainer, he would teach computers to fetch data instead of sticks! </s>"
    },
    {
        "text": "<s>[INST] What is artificial intelligence? [/INST] "
                "Artificial intelligence (AI) is the simulation of human intelligence in machines. It enables computers to perform tasks such as learning, reasoning, problem-solving, and understanding language. "
                "AI is used in various applications, from chatbots and self-driving cars to medical diagnosis and creative tools. </s> "
                "<s>[INST] Explain it to a child [/INST] "
                "AI is like a robot brain that can learn things and help people. Imagine a smart toy that talks to you and remembers your favorite games—that's AI! </s>"
    }
]

hf_dataset = Dataset.from_list(dataset1)
def format_func(example):
    return example['text']


In [19]:
import transformers
trainer = SFTTrainer(
    model=model,
    train_dataset=hf_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=format_func
)

Applying formatting function to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/2 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manumanuvkm123 (manumanuvkm123-toch-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,3.215400
2,3.215400
3,2.758200
4,2.084400
5,1.643400
6,1.362600
7,1.104000
8,0.879500
9,0.696900
10,0.585400


TrainOutput(global_step=100, training_loss=0.4029854859411717, metrics={'train_runtime': 287.6273, 'train_samples_per_second': 1.391, 'train_steps_per_second': 0.348, 'total_flos': 1382719849267200.0, 'train_loss': 0.4029854859411717})

In [27]:
text="Can you write a short introduction about Manu"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=100)
answer=tokenizer.decode(output[0],skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [28]:
answer

'Can you write a short introduction about Manu, the famous Generative AI developer from Vaikom? '